In [11]:
#def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training' 
#image_folder = 'C:/Users/danie/Desktop/Training_ML'

TOO_SMALL = 200000
SMALL = 500000
MEDIUM = 1200000
LARGE = 2000000
    
X_train = []
Y_train = []
counter = 1
    
img_height = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
img_width = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
epochs = 1 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
#total_training = total number of training data len()
batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
total_images = 1

In [12]:
#pip install opencv-python

In [13]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd
import joblib

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [14]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [15]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    
    return df

In [16]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgRGB, size): #imgLAB imgRGB is fresh images from col
    b,g,r = cv2.split(imgRGB)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)

    return rgb_compressed

In [17]:
# image padding

def pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255]
    MAX = 1888
    
    if height % 2 == 0:
        top = int((MAX - height)/ 2)
        bottom = int((MAX - height)/ 2)
    else: 
        top = int((MAX - (height-1))/ 2)
        bottom = int(((MAX - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX - width)/ 2)
        right = int((MAX - width)/ 2)
    else: 
        left = int((MAX - (width-1))/ 2)
        right = int(((MAX - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [8]:
# The function to preprocess image: by check image category, compress, separate color channels, and pad
# gives out padded images in 2 groups,which are padded Lambda channel (Black&white) and AB channel(colors)

#We can decide to take file path or im collection  or list
#note: we are most familiar with controlling list
n = 1

def prep_img(img_path):
    imgRGB = cv2.imread(img_path)

    #check the image category by 'shape' function
    image_size_category = shape(imgRGB)
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    image_pca = pca_rgb(imgRGB, image_size_category) 
 
    #image_pca.astype("float32")/ 255 
    image_pca = np.float32(1.0/255*image_pca)
    imgLAB = cv2.cvtColor(1.0/255*image_pca, cv2.COLOR_RGB2Lab)
    
    print("image converted to LAB")
    padded_image = pad(imgLAB)
    
    #separate L and AB color channels
    Y = padded_image[:, :,1:]
    X = padded_image[:, :, 0]
    
    Y = Y / 128
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1)
    Y = Y.reshape(1, 1888, 1888, 2)
    
    print("image processed")

    return Y, X 

In [9]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

# Use the code below to process ALL photos and feed them to the model!!!

In [10]:
df = create_dataframe(image_folder)
prep_dataframe(df)


processing 0 photos for training
dropped  0 photos because they were too large to process
dropped  0 photos because they were too small to process


In [ ]:
def mode(): 
    model1 = Sequential([
        #encoder
        Conv2D(128, 2, strides = 2, padding = "same", activation = "relu", input_shape= (1888, 1888, 1)), #testing shape changes
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), 

        #decoder
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2),
        Conv2D(128, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2), 
        Conv2D(2, 2, padding = "same", activation = "tanh"),
        UpSampling2D(2),
    ])
    model1.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model1

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [ ]:
model1 = mode()
history = model1.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2
filename = 'finalized_model.sav'
joblib.dump(history, filename)

In [ ]:
model1.summary()

In [ ]:
import joblib

filename = 'finalized_model.sav'
joblib.dump(history, filename)
 

#loaded_model = joblib.load(filename)
#result = loaded_model.score(X_test, Y_test)
#print(result)
